In [1]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
def get_params_to_update(net, feature_extract):
    params_to_update = net.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    return params_to_update

def make_alexnet(out_features, feature_extract = False):
    net = models.alexnet(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[6].in_features
    net.classifier[6] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

def make_vgg16(out_features, feature_extract = False):
    net = models.vgg16_bn(pretrained=True)
    print(net)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[6].in_features
    net.classifier[6] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

def make_mobilenet_v3(out_features, feature_extract = False):
    net = models.mobilenet_v3_small(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[3].in_features
    net.classifier[3] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

def make_inception_v3(out_features, feature_extract = False):
    net = models.inception_v3(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.AuxLogits.fc.in_features
    net.AuxLogits.fc = nn.Linear(num_ftrs, out_features)
    num_ftrs = net.fc.in_features
    net.fc = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer


In [2]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size, early_stopping, is_inception):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    device = None # Wybiera CUDA jeśli jest dostępne, w przeciwnym wypadku CPU
    # device = "cpu" # Odkomentować jeżeli CUDA nie będzie działać
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,early_stopping = early_stopping,
                                 epochs=epochs, optimizer=optimizer, print_results=True, device=device, is_inception=is_inception)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, batch_size=32, is_inception=False):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net, optimizer = net_generator()
        early_stopping = net_training.EarlyStoppingByAccuracy(patience=10)
        results = train_net_random_dataset_split(net, dataset, epochs,optimizer=optimizer,
                                                 early_stopping=early_stopping, batch_size=batch_size, is_inception=is_inception)
        all_results.append(results)
        net_training.print_final_results(results)
        del optimizer, net
        torch.cuda.empty_cache()
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [3]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]

#transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
transform =  transforms.Compose([
    transforms.Normalize((0, 0, 0), (255, 255, 255)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_inception =  transforms.Compose([
    transforms.Resize([299, 299]),
    transforms.Normalize((0, 0, 0), (255, 255, 255)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)
# inception
#dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform_inception)

def net_generator():
    return make_vgg16(out_features=len(selected_birds), feature_extract = False)

results = cross_validate_net(net_generator, dataset, repeat=3, epochs=100, batch_size=64, is_inception=False)
print_validation_results(results)

Training network 1 ...
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

Epoch 0:
	train loss: 0.7307592635753838
	validation loss: 0.05154324455620491, validation accuracy: 98.28767123287672%
	Elapsed time: 0:07:58.449976
Epoch 1:
	train loss: 0.016744073108956856
	validation loss: 0.042737300622544876, validation accuracy: 98.97260273972603%
	Elapsed time: 0:08:15.340315
Epoch 2:
	train loss: 0.004069120876669188
	validation loss: 0.05406910354552204, validation accuracy: 98.63013698630137%
	Elapsed time: 0:08:11.099016
Epoch 3:
	train loss: 0.003985846835295598
	validation loss: 0.03314670254412579, validation accuracy: 98.63013698630137%
	Elapsed time: 0:08:00.627936
Epoch 4:
	train loss: 0.004188109097779568
	validation loss: 0.011439721993081374, validation accuracy: 99.65753424657534%
	Elapsed time: 0:07:58.289489
Epoch 5:
	train loss: 0.004379616309265649
	validation loss: 0.03462816818221791, validation accuracy: 98.97260273972603%
	Elapsed time: 0:07:54.730642
Epoch 6:
	train loss: 0.0006277850443064686
	validation loss: 0.030746528598135463, vali

In [4]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/vgg16_bn_pretrained.csv")